In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
!sudo apt-get install pv

Reading package lists... Done
Building dependency tree       
Reading state information... Done
pv is already the newest version (1.6.6-1).
0 upgraded, 0 newly installed, 0 to remove and 39 not upgraded.


In [5]:
%%time
!pip install --upgrade --force-reinstall --no-deps  kaggle > /dev/null
!mkdir ~/.kaggle

!cp "/content/drive/MyDrive/Kaggle/kaggle.json" ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

import os

if not os.path.exists("/content/input"):
    !mkdir input
    #!kaggle datasets download -d chiman3se/melspeckkillerbase
    !kaggle datasets download -d chiman3se/melspeckkiller-other
    n_files = !unzip -l melspeckkiller-other.zip | grep .pt | wc -l
    !unzip -o melspeckkiller-other.zip -d /content/input | pv -l -s {n_files[0]} > /dev/null

 98% 947M/970M [00:13<00:00, 89.4MB/s]
100% 970M/970M [00:13<00:00, 74.8MB/s]
14.9k 0:00:17 [ 871 /s] [                  <=>                                 ]
CPU times: user 299 ms, sys: 82.3 ms, total: 381 ms
Wall time: 34.9 s


In [9]:
!mkdir working
%cd working

/content/working


In [10]:
import os
import re
import gc

import random
import numpy as np
import pandas as pd
import time
from  ast import literal_eval

import datetime
from datetime import datetime, timedelta, timezone

import librosa
import librosa.display
import matplotlib.pyplot as plt
import seaborn as sn
from sklearn import model_selection
from sklearn import preprocessing
import IPython.display as ipd

import IPython.display
from IPython.display import display

import requests
import matplotlib.pyplot as plt

import torch
import torchaudio
import torchaudio.functional as F
import torchaudio.transforms as T

from sklearn.metrics import label_ranking_average_precision_score

import soundfile as sf
from  soundfile import SoundFile
import numpy as np

from tqdm.notebook import tqdm
from pathlib import Path
import joblib, json
from  sklearn.model_selection  import StratifiedKFold

In [11]:
!pip install -q pysndfx SoundFile audiomentations pretrainedmodels efficientnet_pytorch resnest
!pip install efficientnet_pytorch
!pip install torchlibrosa

     |████████████████████████████████| 67 kB 5.3 MB/s 
     |████████████████████████████████| 58 kB 7.2 MB/s 
     |████████████████████████████████| 154 kB 93.8 MB/s 


In [12]:
from torch import nn, optim
from  torch.utils.data import Dataset, DataLoader

from resnest.torch import resnest50

from efficientnet_pytorch import EfficientNet
import pretrainedmodels
import resnest.torch as resnest_torch

from torchlibrosa.augmentation import SpecAugmentation

In [13]:
def seed_everything(seed=42):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
seed_everything()

In [30]:
NUM_CLASSES = 152
SR = 32000
DURATION = 7

MIXUP =False

SPECAUGMENTATION = False
spec_augmenter = SpecAugmentation(time_drop_width=16,time_stripes_num=4,freq_drop_width=8,freq_stripes_num=2,)

MAX_READ_SAMPLES = 3 # Each record will have 10 melspecs at most, you can increase this on Colab with High Memory Enabled

TRAIN_AUDIO_IMAGES_SAVE_ROOT = Path("/content/output")
MYDRIVE = Path("/content/drive/MyDrive/python/kaggle/birdclef-2022/input/")

# For colab
DATA_ROOT = Path("/content/input")
TRAIN_IMAGES_ROOT = Path("/content/input/audio_images")
TRAIN_LABELS_FILE = Path(MYDRIVE/"df_metadata.csv")
TRAIN_LABEL_PATHS = Path(MYDRIVE/"LABEL_IDS.json")

MODEL_ROOT = Path("/content/drive/My Drive/python/kaggle/birdclef-2022/models")
MODEL_ROOT.mkdir(exist_ok=True, parents=True)

In [15]:
!ls $MYDRIVE

df_metadata.csv		  LABEL_IDS.json	 train_metadata.csv
eBird_Taxonomy_v2021.csv  sample_submission.csv


In [16]:
!ls "/content/drive/MyDrive/python/kaggle/birdclef-2022/input/"

df_metadata.csv		  LABEL_IDS.json	 train_metadata.csv
eBird_Taxonomy_v2021.csv  sample_submission.csv


In [17]:
df = pd.read_csv(TRAIN_LABELS_FILE)
df

,primary_label,secondary_labels,type,latitude,longitude,scientific_name,common_name,author,license,rating,time,url,filename,label_id,filepath,frames,sr,duration,fold
0,afrsil1,[],"['call', 'flight call']",12.3910,-1.4930,Euodice cantans,African Silverbill,Bram Piot,Creative Commons Attribution-NonCommercial-Sha...,2.5,08:00,https://www.xeno-canto.org/125458,afrsil1/XC125458.ogg,0,../input/birdclef-2022/train_audio/afrsil1/XC1...,355265,32000,11.102031,3
1,afrsil1,"['houspa', 'redava', 'zebdov']",['call'],19.8801,-155.7254,Euodice cantans,African Silverbill,Dan Lane,Creative Commons Attribution-NonCommercial-Sha...,3.5,08:30,https://www.xeno-canto.org/175522,afrsil1/XC175522.ogg,0,../input/birdclef-2022/train_audio/afrsil1/XC1...,1504653,32000,47.020406,3
2,afrsil1,[],"['call', 'song']",16.2901,-16.0321,Euodice cantans,African Silverbill,Bram Piot,Creative Commons Attribution-NonCommercial-Sha...,4.0,11:30,https://www.xeno-canto.org/177993,afrsil1/XC177993.ogg,0,../input/birdclef-2022/train_audio/afrsil1/XC1...,1720320,32000,53.760000,2
3,afrsil1,[],"['alarm call', 'call']",17.0922,54.2958,Euodice cantans,African Silverbill,Oscar Campbell,Creative Commons Attribution-NonCommercial-Sha...,4.0,11:00,https://www.xeno-canto.org/205893,afrsil1/XC205893.ogg,0,../input/birdclef-2022/train_audio/afrsil1/XC2...,3375438,32000,105.482438,1
4,afrsil1,[],['flight call'],21.4581,-157.7252,Euodice cantans,African Silverbill,Ross Gallardy,Creative Commons Attribution-NonCommercial-Sha...,3.0,16:30,https://www.xeno-canto.org/207431,afrsil1/XC207431.ogg,0,../input/birdclef-2022/train_audio/afrsil1/XC2...,108669,32000,3.395906,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14847,zebdov,[],"['adult', 'song']",22.6499,120.2872,Geopelia striata,Zebra Dove,Jerome Chie-Jen Ko,Creative Commons Attribution-NonCommercial-Sha...,2.5,08:36,https://www.xeno-canto.org/629769,zebdov/XC629769.ogg,151,../input/birdclef-2022/train_audio/zebdov/XC62...,1953792,32000,61.056000,1
14848,zebdov,[],"['adult', 'sex uncertain', 'song']",1.4168,103.7287,Geopelia striata,Zebra Dove,Kim Chuah Lim,Creative Commons Attribution-NonCommercial-Sha...,4.0,08:00,https://www.xeno-canto.org/642415,zebdov/XC642415.ogg,151,../input/birdclef-2022/train_audio/zebdov/XC64...,475392,32000,14.856000,3
14849,zebdov,[],"['adult', 'call', 'sex uncertain']",1.4347,103.7043,Geopelia striata,Zebra Dove,Kim Chuah Lim,Creative Commons Attribution-NonCommercial-Sha...,4.0,09:00,https://www.xeno-canto.org/665873,zebdov/XC665873.ogg,151,../input/birdclef-2022/train_audio/zebdov/XC66...,1458432,32000,45.576000,2
14850,zebdov,[],['song'],3.3508,101.2451,Geopelia striata,Zebra Dove,Jelle Scharringa,Creative Commons Attribution-NonCommercial-Sha...,2.5,11:15,https://www.xeno-canto.org/666194,zebdov/XC666194.ogg,151,../input/birdclef-2022/train_audio/zebdov/XC66...,689632,32000,21.551000,4


In [20]:
###TRAIN_BATCH_SIZE = 100
TRAIN_NUM_WORKERS = 1

###VAL_BATCH_SIZE = 128
VAL_NUM_WORKERS = 1

TRAIN_BATCH_SIZE = 8
VAL_BATCH_SIZE = 16

DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

print("Device:", DEVICE)

Device: cuda


In [29]:
!ls /content/input/audio_images

afrsil1  bongul   chbsan   gamqua   houspa   madpet   osprey   ribgul	sora
akekee	 brant	  chemun   glwgul   hudgod   magpet1  pagplo   rinduc	spodov
akepa1	 brnboo   chukar   gnwtea   iiwi     mallar3  palila   rinphe	sposan
akiapo	 brnnod   cintea   golphe   incter1  masboo   parjae   rocpig	towsol
akikik	 brnowl   comgal1  grbher3  jabwar   mauala   pecsan   rorpar	wantat1
amewig	 brtcur   commyn   grefri   japqua   maupar   peflov   rudtur	warwhe1
aniani	 bubsan   compea   gresca   kalphe   merlin   perfal   ruff	wesmea
apapan	 buffle   comsan   gryfra   kauama   mitpar   pibgre   saffin	wessan
arcter	 bulpet   comwax   gwfgoo   laugul   moudov   pomjae   sander	wetshe
barpet	 burpar   coopet   hawama   layalb   norcar   puaioh   semplo	whfibi
bcnher	 buwtea   crehon   hawcoo   lcspet   norhar2  reccar   sheowl	whiter
belkin1  cacgoo1  dunlin   hawcre   leasan   normoc   redava   shtsan	whttro
bkbplo	 calqua   elepai   hawgoo   leater1  norpin   redjun   skylar	wiltur
bknsti	 cango

In [35]:
def get_df(train_lables=TRAIN_LABELS_FILE, train_label_paths=TRAIN_LABEL_PATHS):
    df = None
    LABEL_IDS = {}
        
    df = pd.read_csv(train_lables)
    #df["impath"] = df.apply(lambda row: TRAIN_IMAGES_ROOT/"{}/{}.pt".format(row.filename), axis=1) ### npyのファイルpathカラム
    
    #df["impath"] = df["filename"].apply(lambda x: str(TRAIN_IMAGES_ROOT) +"/"+ x[:-4] + ".pt" )
    df["impath"] = df["filename"].apply(lambda x: str(TRAIN_IMAGES_ROOT) +"/"+ x +".npy" )
    
    df["secondary_labels"] = df["secondary_labels"].apply(literal_eval)     ### from  ast import literal_evalくそ便利.
                                                                            ### そのまま読むと、second label の ["xxxx","yyyy"]の[や"はstrとして認識されてしまう。 文字列が格納されているリストとしてほしいのにー！を解消してくれる.
    
    with open(str(train_label_paths)) as f:
        LABEL_IDS.update(json.load(f))

    return LABEL_IDS, df

In [37]:
LABEL_IDS, df = get_df(train_label_paths=TRAIN_LABEL_PATHS)

In [42]:
MAX_READ_SAMPLES = 2
def load_data(df):
    def load_row(row):
        # impath = TRAIN_IMAGES_ROOT/f"{row.primary_label}/{row.filename}.npy"
        return row.filename, np.load(str(row.impath))[:MAX_READ_SAMPLES]    ### MAX_READ_SAMPELES=5　これで区切ったメルの上限を設定。 わざわざ絞らずに全部学習させてもいいような気がする
    #pool = joblib.Parallel(4)
    pool = joblib.Parallel(2)
    mapper = joblib.delayed(load_row)
    tasks = [mapper(row) for row in df.itertuples(False)]
    res = pool(tqdm(tasks))
    res = dict(res)
    return res

In [43]:
audio_image_store = load_data(df)
len(audio_image_store)

  0%|          | 0/14852 [00:00<?, ?it/s]

14852

In [47]:
for k,v in audio_image_store.items():
    print(k)
    print(v.shape)
    break

afrsil1/XC125458.ogg
(1, 128, 281)


In [48]:
def get_model(name, num_classes=NUM_CLASSES):
    """
    Loads a pretrained model. 
    Supports ResNest, ResNext-wsl, EfficientNet, ResNext and ResNet.

    Arguments:
        name {str} -- Name of the model to load

    Keyword Arguments:
        num_classes {int} -- Number of classes to use (default: {1})

    Returns:
        torch model -- Pretrained model
    """
    if "resnest" in name:
        model = getattr(resnest_torch, name)(pretrained=True)
    elif "wsl" in name:
        model = torch.hub.load("facebookresearch/WSL-Images", name)
    elif name.startswith("resnext") or  name.startswith("resnet"):
        model = torch.hub.load("pytorch/vision:v0.6.0", name, pretrained=True)
    elif name.startswith("tf_efficientnet_b"):
        model = getattr(timm.models.efficientnet, name)(pretrained=True)
    elif "efficientnet-b" in name:
        ### EfficientNet　の 
        #model = EfficientNet.from_pretrained(name)
        model = EfficientNet.from_pretrained(name)
    else:
        model = pretrainedmodels.__dict__[name](pretrained='imagenet')



    ### クラス属性 ドット . でアクセスできるメンバー変数やメンバー関数
    ### hasattrであるかどうかをちぇっくして

    if hasattr(model, "fc"):
        nb_ft = model.fc.in_features
        model.fc = nn.Linear(nb_ft, num_classes)
    elif hasattr(model, "_fc"):
        nb_ft = model._fc.in_features
        model._fc = nn.Linear(nb_ft, num_classes)
    elif hasattr(model, "classifier"):
        nb_ft = model.classifier.in_features
        model.classifier = nn.Linear(nb_ft, num_classes)
    elif hasattr(model, "last_linear"):
        nb_ft = model.last_linear.in_features
        model.last_linear = nn.Linear(nb_ft, num_classes)

    return model

In [70]:
class BirdClefDataset(Dataset):
    def __init__(self,  audio_image_store, meta, sr=SR, is_train=True, num_classes=NUM_CLASSES, duration=DURATION):
        
        self.audio_image_store = audio_image_store
        self.meta = meta.copy().reset_index(drop=True)
        self.sr = sr
        self.is_train = is_train
        self.num_classes = num_classes
        self.duration = duration
        self.audio_length = self.duration*self.sr
    
    @staticmethod
    def normalize(image):
        image = image / 255.0
        image = np.stack([image, image, image])
        return image

    def __len__(self):
        return len(self.meta)
    
    def __getitem__(self, idx):
        row = self.meta.iloc[idx]
        image = self.audio_image_store[row.filename]
        
        #print(row.impath)
        #image = torch.load(row.impath)

        #image = image[np.random.choice(len(image))]        ### 中山 5秒メルスペクトログラムは 1音源につき5個あるが、random でしかとってきてない。
        image = image[0]        ### 中山 5秒メルスペクトログラムは 1音源につき5個あるが、random でしかとってきてない。
        #image = image.permute(2, 0, 1)
        image = self.normalize(image)
        #print(image.shape)
        
        ### 281を201へと変換 7秒から5秒へ
        secbin = 40
        if image.shape[2] == 281:
            image = image[:,:,secbin:-secbin]   ### 281を201へと変換 7秒から5秒へ        

        # Label smoothing
        t = np.zeros(self.num_classes, dtype=np.float32) + 0.0025 
        t[row.label_id] = 0.995       
        
        return image, t

In [71]:
train_data = BirdClefDataset(audio_image_store,meta=df, sr=SR, duration=DURATION, is_train=True)
len(train_data)

train_laoder = DataLoader(train_data, batch_size=TRAIN_BATCH_SIZE, num_workers=TRAIN_NUM_WORKERS, shuffle=True)

In [72]:
train_data.__getitem__(3500)[0].shape

(3, 128, 201)

In [73]:
train_laoder = tqdm(train_laoder, leave = False)
epoch_bar = train_laoder

for i,(xb, yb) in enumerate(epoch_bar):
    print(i)
    print("return",xb.shape)
    if i == 10:
        break

  0%|          | 0/1857 [00:00<?, ?it/s]

0
return torch.Size([8, 3, 128, 201])
1
return torch.Size([8, 3, 128, 201])
2
return torch.Size([8, 3, 128, 201])
3
return torch.Size([8, 3, 128, 201])
4
return torch.Size([8, 3, 128, 201])
5
return torch.Size([8, 3, 128, 201])
6
return torch.Size([8, 3, 128, 201])
7
return torch.Size([8, 3, 128, 201])
8
return torch.Size([8, 3, 128, 201])
9
return torch.Size([8, 3, 128, 201])
10
return torch.Size([8, 3, 128, 201])


In [94]:
def one_step( xb, yb, net, criterion, optimizer, scheduler=None):

    xb, yb = xb.to(DEVICE,dtype=torch.float), yb.to(DEVICE)

    optimizer.zero_grad()

    o = net(xb)   ###modelにinputしているとこ
    loss = criterion(o, yb)
    loss.backward()
    optimizer.step()
    
    with torch.no_grad():
        l = loss.item()

        o = o.sigmoid()
        yb = (yb > 0.5)*1.0
        lrap = label_ranking_average_precision_score(yb.cpu().numpy(), o.cpu().numpy())

        o = (o > 0.5)*1.0

        prec = (o*yb).sum()/(1e-6 + o.sum())
        rec = (o*yb).sum()/(1e-6 + yb.sum())
        f1 = 2*prec*rec/(1e-6+prec+rec)

    if scheduler is not None:
        scheduler.step()

    return l, lrap, f1.item(), rec.item(), prec.item()

In [96]:
@torch.no_grad()
def evaluate(net, criterion, val_laoder):
    net.eval()

    os, y = [], []
    val_laoder = tqdm(val_laoder, leave = False, total=len(val_laoder))

    for icount, (xb, yb) in  enumerate(val_laoder):

        y.append(yb.to(DEVICE))

        xb = xb.to(DEVICE,dtype=torch.float)
        o = net(xb)

        os.append(o)

    y = torch.cat(y)
    o = torch.cat(os)

    l = criterion(o, y).item()
    
    o = o.sigmoid()
    y = (y > 0.5)*1.0

    lrap = label_ranking_average_precision_score(y.cpu().numpy(), o.cpu().numpy())

    o = (o > 0.5)*1.0

    prec = ((o*y).sum()/(1e-6 + o.sum())).item()
    rec = ((o*y).sum()/(1e-6 + y.sum())).item()
    f1 = 2*prec*rec/(1e-6+prec+rec)

    return l, lrap, f1, rec, prec, 
    

In [76]:
def one_epoch(net, criterion, optimizer, scheduler, train_laoder, val_laoder):
    net.train()
    l, lrap, prec, rec, f1, icount = 0.,0.,0.,0., 0., 0
    train_laoder = tqdm(train_laoder, leave = False)
    epoch_bar = train_laoder
    
    for (xb, yb) in  epoch_bar:
        # epoch_bar.set_description("----|----|----|----|---->")

        if SPECAUGMENTATION == True:
            xb = spec_augmenter(xb)

        _l, _lrap, _f1, _rec, _prec = one_step(xb, yb, net, criterion, optimizer)
        l += _l
        lrap += _lrap
        f1 += _f1
        rec += _rec
        prec += _prec

        icount += 1
            
        if hasattr(epoch_bar, "set_postfix") and not icount%10:
            epoch_bar.set_postfix(
                loss="{:.6f}".format(l/icount),
                lrap="{:.3f}".format(lrap/icount),
                prec="{:.3f}".format(prec/icount),
                rec="{:.3f}".format(rec/icount),
                f1="{:.3f}".format(f1/icount),
            )
    
    scheduler.step()

    l /= icount
    lrap /= icount
    f1 /= icount
    rec /= icount
    prec /= icount
    
    l_val, lrap_val, f1_val, rec_val, prec_val = evaluate(net, criterion, val_laoder)
    
    return (l, l_val), (lrap, lrap_val), (f1, f1_val), (rec, rec_val), (prec, prec_val)

In [77]:
class AutoSave:
    def __init__(self, top_k=2, metric="f1", mode="min", root=None, name="ckpt"):
        self.top_k = top_k
        self.logs = []
        self.metric = metric
        self.mode = mode
        self.root = Path(root or MODEL_ROOT)
        assert self.root.exists()
        self.name = name

        self.top_models = []
        self.top_metrics = []

    def log(self, model, metrics):

        metric = metrics[self.metric]
        rank = self.rank(metric)

        self.top_metrics.insert(rank+1, metric)
        if len(self.top_metrics) > self.top_k:
            self.top_metrics.pop(0)

        self.logs.append(metrics)
        self.save(model, metric, rank, metrics["epoch"])


    def save(self, model, metric, rank, epoch):

        name = "{}_epoch_{:02d}_{}_{:.04f}".format(self.name, epoch, self.metric, metric)
        name = re.sub(r"[^\w_-]", "", name) + ".pth"
        path = self.root.joinpath(name)

        print("savefunc",path)

        old_model = None
        self.top_models.insert(rank+1, name)
        if len(self.top_models) > self.top_k:
            old_model = self.root.joinpath(self.top_models[0])
            self.top_models.pop(0)      

        torch.save(model.state_dict(), path.as_posix())

        if old_model is not None:
            old_model.unlink()

        self.to_json()


    def rank(self, val):
        r = -1
        for top_val in self.top_metrics:
            if val <= top_val:
                return r
            r += 1

        return r
    
    def to_json(self):
        # t = time.strftime("%Y%m%d%H%M%S")
        name = "{}_logs".format(self.name)
        name = re.sub(r"[^\w_-]", "", name) + ".json"
        path = self.root.joinpath(name)

        with path.open("w") as f:
            json.dump(self.logs, f, indent=2)


In [78]:
def one_fold(model_name, fold, train_set, val_set, epochs=20, save=True, save_root=None):

    save_root = Path(save_root) or MODEL_ROOT

    print(save_root)

    saver = AutoSave(root=save_root, name=f"birdclef_{model_name}_fold{fold}", metric="f1_val")

    net = get_model(model_name).to(DEVICE)

    criterion = nn.BCEWithLogitsLoss()

    optimizer = optim.Adam(net.parameters(), lr=8e-4)
    scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, eta_min=1e-5, T_max=epochs)

    train_data = BirdClefDataset(audio_image_store, meta=df.iloc[train_set].reset_index(drop=True),
                            sr=SR, duration=DURATION, is_train=True)
    #train_laoder = DataLoader(train_data, batch_size=TRAIN_BATCH_SIZE, num_workers=TRAIN_NUM_WORKERS, shuffle=True, pin_memory=True)  ### 中山追記 drop_last
    train_laoder = DataLoader(train_data, batch_size=TRAIN_BATCH_SIZE, num_workers=TRAIN_NUM_WORKERS, shuffle=True, pin_memory=True, drop_last=True)

    val_data = BirdClefDataset(audio_image_store, meta=df.iloc[val_set].reset_index(drop=True),  sr=SR, duration=DURATION, is_train=False)
    #val_laoder = DataLoader(val_data, batch_size=VAL_BATCH_SIZE, num_workers=VAL_NUM_WORKERS, shuffle=False)  ### 中山追記 drop_last
    val_laoder = DataLoader(val_data, batch_size=VAL_BATCH_SIZE, num_workers=VAL_NUM_WORKERS, shuffle=False, drop_last=True)

    epochs_bar = tqdm(list(range(epochs)), leave=False)
    for epoch  in epochs_bar:
        epochs_bar.set_description(f"--> [EPOCH {epoch:02d}]")
        net.train()

        (l, l_val), (lrap, lrap_val), (f1, f1_val), (rec, rec_val), (prec, prec_val) = one_epoch(
            net=net,
            criterion=criterion,
            optimizer=optimizer,
            scheduler=scheduler,
            train_laoder=train_laoder,
            val_laoder=val_laoder,
        )

        epochs_bar.set_postfix(
        loss="({:.6f}, {:.6f})".format(l, l_val),
        prec="({:.3f}, {:.3f})".format(prec, prec_val),
        rec="({:.3f}, {:.3f})".format(rec, rec_val),
        f1="({:.3f}, {:.3f})".format(f1, f1_val),
        lrap="({:.3f}, {:.3f})".format(lrap, lrap_val),
        )

        print(
            "[{epoch:02d}] loss: {loss} lrap: {lrap} f1: {f1} rec: {rec} prec: {prec}".format(
                epoch=epoch,
                loss="({:.6f}, {:.6f})".format(l, l_val),
                prec="({:.3f}, {:.3f})".format(prec, prec_val),
                rec="({:.3f}, {:.3f})".format(rec, rec_val),
                f1="({:.3f}, {:.3f})".format(f1, f1_val),
                lrap="({:.3f}, {:.3f})".format(lrap, lrap_val),
            )
        )

        if save:
            metrics = {
                "loss": l, "lrap": lrap, "f1": f1, "rec": rec, "prec": prec,
                "loss_val": l_val, "lrap_val": lrap_val, "f1_val": f1_val, "rec_val": rec_val, "prec_val": prec_val,
                "epoch": epoch,
            }

            saver.log(net, metrics)

In [97]:
def train(model_name, epochs=20, save=True, n_splits=5, seed=177, save_root=None, suffix="", folds=None):
    gc.collect()
    torch.cuda.empty_cache()

    if SPECAUGMENTATION:
        augmantation_name = "specaugmentation"
    else :
        augmantation_name = "None"


    
    save_root = save_root or MODEL_ROOT/f"{model_name}{suffix}"
    save_root.mkdir(exist_ok=True, parents=True)

    fold_bar = tqdm(df.reset_index().groupby("fold").index.apply(list).items(), total=df.fold.max()+1)
    
    for fold, val_set in fold_bar:
        if folds and not fold in folds:
            continue
        
        print(f"\n############################### [FOLD {fold}]")
        fold_bar.set_description(f"[FOLD {fold}]")
        train_set = np.setdiff1d(df.index, val_set)
            
        one_fold(model_name, fold=fold, train_set=train_set , val_set=val_set , epochs=epochs, save=save, save_root=save_root)
        
        gc.collect()
        torch.cuda.empty_cache()

In [80]:

save_root = Path("/content/drive/My Drive/python/kaggle/birdclef-2022/models/efficientnet-b7_sr32000_d7_librosamelspec/")
save_root.mkdir(exist_ok=True, parents=True)


birdclef_efficientnet-b7_fold0_epoch_01_f1_val_05036_20220402065205.pth
birdclef_efficientnet-b7_fold0_epoch_02_f1_val_05343_20220402065934.pth
birdclef_efficientnet-b7_fold0_logs.json


In [81]:
MODEL_NAMES = [
    "efficientnet-b7",
]

In [98]:
### fold 3,4,5
epochs = 25

JST = timezone(timedelta(hours=+9), 'JST')
now = datetime.now(JST)
current_time = now.strftime("%Y-%m-%d-%H-%M", )

for model_name in MODEL_NAMES:
    print("\n\n###########################################", model_name.upper())
    try:
        train(model_name, epochs=epochs, save=True, suffix=f"_sr{SR}_d{DURATION}_librosamel{current_time}",folds=[3,4])
    except Exception as e:
        # print(f"Error {model_name} : \n{e}")
        raise ValueError() from  e



########################################### EFFICIENTNET-B7


  0%|          | 0/5 [00:00<?, ?it/s]


############################### [FOLD 3]
/content/drive/My Drive/python/kaggle/birdclef-2022/models/efficientnet-b7_sr32000_d7_librosamel2022-04-16-10-49
Loaded pretrained weights for efficientnet-b7


  0%|          | 0/25 [00:00<?, ?it/s]

  0%|          | 0/1485 [00:00<?, ?it/s]

  0%|          | 0/185 [00:00<?, ?it/s]

[00] loss: (0.053807, 0.048361) lrap: (0.125, 0.164) f1: (0.000, 0.011) rec: (0.002, 0.006) prec: (0.000, 0.362)
savefunc /content/drive/My Drive/python/kaggle/birdclef-2022/models/efficientnet-b7_sr32000_d7_librosamel2022-04-16-10-49/birdclef_efficientnet-b7_fold3_epoch_00_f1_val_00113.pth


  0%|          | 0/1485 [00:00<?, ?it/s]

KeyboardInterrupt: ignored

In [ ]:
%cd /content/

In [ ]:

kaggle_savepath = "/content/drive/MyDrive/python/kaggle/birdclef-2022/models/efficientnet-b7_sr32000_d7_v1_v12022-04-02-16-08"


!kaggle datasets init -p $kaggle_savepath

import json
from collections import OrderedDict
import re

with open(f'{kaggle_savepath}/dataset-metadata.json') as f:
    df = json.load(f)

expname = "kkiller-base"
datasettitle = "efficientnetb7-" + expname
datasetid = re.sub(r"[^a-zA-Z0-9]","",datasettitle)

df["title"] = datasettitle
df["id"] = df["id"].split("/")[0] +"/" +datasetid

with open(f'{kaggle_savepath}/dataset-metadata.json', mode='w') as f:
    json.dump(df, f, indent=2)

!kaggle datasets create -p $kaggle_savepath

Data package template written to: /content/drive/MyDrive/python/kaggle/birdclef-2022/models/efficientnet-b7_sr32000_d7_v1_v12022-04-02-16-08/dataset-metadata.json
Starting upload for file birdclef_efficientnet-b7_fold3_logs.json
100% 8.82k/8.82k [00:03<00:00, 2.45kB/s]
Upload successful: birdclef_efficientnet-b7_fold3_logs.json (9KB)
Starting upload for file birdclef_efficientnet-b7_fold0_epoch_24_f1_val_07120_20220326230023.pth
100% 246M/246M [00:09<00:00, 27.1MB/s]
Upload successful: birdclef_efficientnet-b7_fold0_epoch_24_f1_val_07120_20220326230023.pth (246MB)
Starting upload for file birdclef_efficientnet-b7_fold2_epoch_24_f1_val_07044_20220327123329.pth
100% 246M/246M [00:09<00:00, 28.2MB/s]
Upload successful: birdclef_efficientnet-b7_fold2_epoch_24_f1_val_07044_20220327123329.pth (246MB)
Starting upload for file birdclef_efficientnet-b7_fold1_epoch_24_f1_val_07111_20220327054724.pth
100% 246M/246M [00:09<00:00, 27.7MB/s]
Upload successful: birdclef_efficientnet-b7_fold1_epoch_24